In [1]:
# Clean fix for dataset loading issues (common error with "**" pattern in fsspec)
!pip uninstall -y gcsfs
!pip install --upgrade fsspec==2023.6.0


Found existing installation: gcsfs 2025.3.2
Uninstalling gcsfs-2025.3.2:
  Successfully uninstalled gcsfs-2025.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires gcsfs!=2025.5.0,>=2023.3.0, which is not installed.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 req

In [ ]:
import os
os.kill(os.getpid(), 9)


In [2]:
!pip install transformers==4.38.2 accelerate datasets rouge-score --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━

In [1]:
from datasets import load_dataset

# Load CNN/DailyMail test set
raw_dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
subset = raw_dataset.select(range(50))  # First 50 samples

# Form 5 multi-document samples of 10 docs each (concatenated)
multidoc_test = []
for i in range(0, 50, 10):
    docs = subset[i:i+10]
    joined_docs = " ".join(docs["article"])
    joined_summary = " ".join(docs["highlights"])
    multidoc_test.append({
        "documents": joined_docs,
        "summary": joined_summary
    })

print("✅ Created", len(multidoc_test), "long-sequence samples.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

✅ Created 5 long-sequence samples.


In [12]:
import torch

def summarize_with_long_model(model, tokenizer, documents, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.to(device)
    model.eval()

    # Join all docs and truncate to max length accepted by tokenizer
    input_text = " ".join(documents)
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=4096).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=512,
            num_beams=4,
            early_stopping=True,
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [13]:
from transformers import LEDTokenizer, LEDForConditionalGeneration

led_tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")
led_model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
def summarize_with_led_v2(docs, tokenizer, model, device="cpu"):
    input_text = "summarize: " + " ".join(docs)  # Add summarization prompt (important for LED)
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=16384
    ).to(device)

    # LED requires global attention on at least 1 token — typically the first
    global_attention_mask = torch.zeros_like(inputs["input_ids"])
    global_attention_mask[:, 0] = 1  # <s> token gets global attention

    with torch.no_grad():
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            global_attention_mask=global_attention_mask,
            num_beams=4,
            max_length=512,  # you can increase to 1024 if GPU allows
            early_stopping=True,
            no_repeat_ngram_size=3,
            length_penalty=2.0
        )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


generated_led = []
device = "cuda" if torch.cuda.is_available() else "cpu"

for item in multidoc_test:
    summary = summarize_with_led_v2(item["documents"], led_tokenizer, led_model, device=device)
    generated_led.append({
        "reference": item["summary"],
        "generated": summary
    })

print("✅ LED Summaries Generated:", len(generated_led))


✅ LED Summaries Generated: 5


In [17]:
from rouge_score import rouge_scorer
# Check if summaries are still empty
for i, x in enumerate(generated_led):
    if not x["generated"].strip():
        print(f"⚠️ Summary {i} is empty")

def compute_rouge_scores(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        scores["rouge1"].append(score["rouge1"].fmeasure)
        scores["rouge2"].append(score["rouge2"].fmeasure)
        scores["rougeL"].append(score["rougeL"].fmeasure)

    avg_scores = {k: sum(v)/len(v) for k, v in scores.items()}
    return avg_scores

# ✅ Run ROUGE on LED (Longformer) outputs
rouge_scores_led = compute_rouge_scores(
    [x["generated"] for x in generated_led],
    [x["reference"] for x in generated_led]
)

print("🔍 ROUGE-1:", rouge_scores_led["rouge1"])
print("🔍 ROUGE-2:", rouge_scores_led["rouge2"])
print("🔍 ROUGE-L:", rouge_scores_led["rougeL"])


🔍 ROUGE-1: 0.07208152864526365
🔍 ROUGE-2: 0.002909066846874605
🔍 ROUGE-L: 0.050345842354566996
